# Coupling rainfall-runoff-soil moisture in Landlab
In this tutorial we couple 3 Landlab components in order to answer 2 research questions.
The coupling of `SpatialPrecipitationDistribution` and `OverlandFlow` is based of the tutorial **Coupled rainfall runoff model with OverlandFlow**, on which we couple `infiltrate_soil_green_ampt`




## Q1 - How does heterogeneous soil moisture and rainfall events impact stream discharge?

## Q2 - How does different rainfall events regulates soil moisture?

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>